# Senator Filings Analysis

***

## Imports

In [ ]:
from collections import defaultdict
import datetime as dt
from functools import lru_cache
import matplotlib.pyplot as plt
import numpy as np
from os import path
import pickle
import pandas as pd
import yfinance as yf

## Introduction

In this notebook, we explore stock orders that were publicly filed by U.S. senators. The filings are scraped from https://efdsearch.senate.gov/search/. We calculate the returns of each senator by mimicking their buys and sells.

***

## Loading data

The `senators.pickle` file is scraped using the script in the root of the repository.

In [ ]:
with open('senators.pickle', 'rb') as f:
    senators_tx = pickle.load(f)

## Data cleaning

### Filling in missing tickers

In this section, we fill in as many of the missing ticker symbols as we can. 

In [ ]:
def tokenize(asset_name):
    """ Convert an asset name into useful tokens. """
    token_string = asset_name\
        .replace('(', '')\
        .replace(')', '')\
        .replace('-', ' ')\
        .replace('.', '')
    return token_string.split(' ')

def token_is_ticker(token, token_blacklist):
    return len(token) <= 4 and token.upper() not in token_blacklist

# These generic words do not help us determine the ticker
blacklist = {
    "1",
    "180",
    "21ST",
    "66",
    "A",
    "AB",
    "ADR",
    "ATK",
    "B",
    "BANK",
    "C",
    "CMN",
    "CO",
    "COM",
    "CORP",
    "DEL",
    "E",
    "ETF",
    "FD",
    "FUND",
    "GROW",
    "GRP",
    "HACK",
    "HLDG",
    "HOME",
    "INC",
    "INTL",
    "INV",
    "IRON",
    "LABS",
    "LP",
    "LTD",
    "MDA",
    "N/L",
    "NEW",
    "PFD",
    "PLC",
    "PUT",
    "SA",
    "SD",
    "SE",
    "SHRS",
    "SHS",
    "SPDR",
    "TD",
    "TF",
    "THE",
    "TR",
    "UNIT",
    "US",
    "VCA",
    "WALT"
}

missing_tickers = set(senators_tx[
    (senators_tx['ticker'] == '--')
    | (senators_tx['ticker'] == '')
]['asset_name'])

ticker_map = {}
unmapped_tickers = set()
for m in missing_tickers:
    tokens = tokenize(m)
    if token_is_ticker(tokens[0], blacklist):
        ticker_map[m] = tokens[0].upper()
        continue
    elif token_is_ticker(tokens[-1], blacklist):
        ticker_map[m] = tokens[-1].upper()
        continue
    unmapped_tickers.add(m)

As a second pass, we assign tickers to asset names that have any of the specified keywords.

In [ ]:
phrase_to_ticker = {
    'FOX': 'FOX',
    'AMAZON': 'AMZN',
    'AARON': 'AAN',
    'ALTRIA': 'MO',
    'APPLE': 'AAPL',
    'CHEVRON': 'CVX',
    'DUPONT': 'DD',
    'ALPHABET': 'GOOGL',
    'GOOG': 'GOOGL',
    'GENERAL ELECTRIC': 'GE',
    'JOHNSON': 'JNJ',
    'NEWELL': 'NWL',
    'OWENS': 'OMI',
    'PFIZER': 'PFE',
    'TYSON': 'TSN',
    'UNDER ARMOUR': 'UAA',
    'VERIZON': 'VZ',
    'WALT': 'DIS'
}

for m in unmapped_tickers:
    for t in phrase_to_ticker:
        if t in m.upper():
            ticker_map[m] = phrase_to_ticker[t]

clean_senators = senators_tx.copy()
for a, t in ticker_map.items():
    clean_senators.loc[clean_senators['asset_name'] == a, 'ticker'] = t

### Filtering rows and columns

We filter out useless rows and missing symbols, and then add some useful columns for the final dataset.

In [ ]:
clean_senators = clean_senators[clean_senators['ticker'] != '--']
clean_senators = clean_senators.assign(
    ticker=clean_senators['ticker'].map(
        lambda s: s.replace('--', '').replace('\n', '')))

clean_senators = clean_senators[clean_senators['order_type'] != 'Exchange']

In [ ]:
def parse_tx_amount(amt):
    """ Get the lower bound for the transaction amount. """
    try:
        return int(amt.replace('Over $50,000,000', '50000000')
                   .split(' - ')[0]
                   .replace(',' ,'')
                   .replace('$' ,''))
    except:
        return 0

clean_senators = clean_senators.assign(
    tx_estimate=clean_senators['tx_amount'].map(parse_tx_amount))
full_name =\
    clean_senators['first_name'] + ' ' + clean_senators['last_name']
clean_senators = clean_senators.assign(full_name=full_name)
useful_cols = [
    'file_date',
    'tx_date',
    'full_name',
    'order_type',
    'ticker',
    'tx_estimate'
]
clean_senators = clean_senators[useful_cols]
clean_senators = clean_senators.assign(
    tx_date=clean_senators['tx_date'].map(
        lambda v: dt.datetime.strptime(v, '%m/%d/%Y')))
clean_senators = clean_senators.assign(
    file_date=clean_senators['file_date'].map(
        lambda v: dt.datetime.strptime(v, '%m/%d/%Y')))

## Returns calculation

These cells help us download the market data for the specified tickers. We store the market data in files so we don't need to repeatedly download the same information.

In [ ]:
def download_for_ticker(ticker, cache_check=True):
    if cache_check and path.exists('stocks/{0}.pickle'.format(ticker)):
        return
    d = yf.Ticker(ticker)
    with open('stocks/{0}.pickle'.format(ticker), 'wb') as f:
        shares = 1
        try:
            shares = d.info['sharesOutstanding']
        except:
            # Swallow missing shares outstanding
            pass
        pickle.dump({
            'price': d.history(period="max").reset_index(),
            'shares': shares
        }, f)

def load_for_ticker(ticker):
    with open('stocks/{0}.pickle'.format(ticker), 'rb') as f:
        dump = pickle.load(f)
    raw = dump['price']
    raw = raw.assign(market_cap=raw['Close'] * dump['shares'])
    return raw[['Date', 'Close', 'market_cap']]\
        .rename(columns={'Date': 'date', 'Close': 'price'})

def download_spy() -> None:
    download_for_ticker('SPY')

In [ ]:
all_tickers = set(clean_senators['ticker'])
for i, t in enumerate(all_tickers):
    if i % 100 == 0:
        print('Working on ticker {0}'.format(i))
    try:
        download_for_ticker(t)
    except Exception as e:
        print('Ticker {0} failed with exception: {1}'.format(t, e))

### Mimicking buy + sell orders

We calculate a given senator's return by calculating the return between each buy or sell order, and then solving for the cumulative return. We convert that to a CAGR given the time period the senator was investing.

We keep track of how many units of each stock a senator is holding. If we ever see a filing that indicates the senator sold more than we estimated they are holding, we just sell all of the units we have on record. (We do not allow the senator to go short.)

In [ ]:
buckets = [
    (1000, 15000),
    (15000, 50000),
    (50000, 100000),
    (100000, 250000),
    (250000, 500000),
    (500000, 1000000),
    (1000000, 5000000),
    (5000000, 25000000),
    (25000000, 50000000),
    (50000000, float('inf'))
]

def same_bucket(a, b):
    """
    If the value of the stock units is roughly the same, sell all units
    """
    for idx1, (v1, v2) in enumerate(buckets):
        if a >= v1 and a < v2:
            break
    for idx2, (v1, v2) in enumerate(buckets):
        if b >= v1 and b < v2:
            break
    return idx1 == idx2

def _price_for_date(df, date):
    df = df[df['date'] >= date].sort_values(by='date')
    return df['price'].iloc[0]

@lru_cache(maxsize=128)
def ticker_at_date(ticker, date):
    """ Price of a ticker at a given date. """
    try:
        data = load_for_ticker(ticker)
        # Sell at the next opportunity possible
        return _price_for_date(data, date)
    except:
        download_for_ticker(ticker, cache_check=False)
        data = load_for_ticker(ticker)
        return _price_for_date(data, date)

def portfolio_value(stocks, date):
    """ Value of a portfolio assuming each ticker has the specified
    number of units. """
    v = 0
    for s, units in stocks.items():
        if units == 0:
            continue
        try:
            v += ticker_at_date(s, date) * units
        except Exception as e:
            print('Error calculating portfolio value: {}'.format(e))
    return v

def calculate_return(before_values,
                     after_values,
                     begin_date,
                     end_date,
                     tx_dates):
    """ Calculate useful metrics given the senators portfolio value
    over time. """
    before_values.pop(0)
    after_values.pop(-1)
    # We calculate the total return by calculating the return
    # between each transaction, and solving for the cumulative
    # return.
    growth = np.array(before_values) / np.array(after_values)
    portfolio_return = np.prod(growth[~np.isnan(growth)])
    years = (end_date - begin_date).days / 365
    if years == 0 or portfolio_return == 0:
        cagr = 0
    else:
        cagr = portfolio_return**(1 / years)
    # Cumulative return calculation
    tx_dates.pop(0)
    tx_dates = np.array(tx_dates)
    tx_dates = tx_dates[~np.isnan(growth)]
    cum_growth = np.cumprod(growth[~np.isnan(growth)])
    growth_df = pd.DataFrame([
        {
            'date': tx_dates[i],
            'cum_growth': cum_growth[i]
        } for i in range(len(cum_growth))
    ])
    return {
        'begin_date': begin_date,
        'end_date': end_date,
        'portfolio_return': portfolio_return,
        'annual_cagr': cagr,
        'growth': growth_df
    }

def return_for_senator(rows, date_col='tx_date'):
    stocks = defaultdict(int)
    # Value of portfolio at various timepoints to calculate return
    portfolio_value_before_tx = []
    portfolio_value_after_tx = []
    tx_dates = []
    rows = rows.sort_values(by=date_col)
    for _, row in rows.iterrows():
        date = row[date_col]
        if date_col == 'file_date':
            # We can't execute the trade the same day
            date += dt.timedelta(days=1)
        try:
            value_before_tx = portfolio_value(stocks, date)
            if 'Purchase' in row['order_type']:
                tx_amt = row['tx_estimate']
                n_units = tx_amt / ticker_at_date(row['ticker'], date)
                stocks[row['ticker']] += n_units
            elif 'Sale' in row['order_type']:
                stock_price = ticker_at_date(row['ticker'], date)
                current_value = stock_price * stocks[row['ticker']]
                if 'Full' in row['order_type'] or\
                    same_bucket(row['tx_estimate'], current_value):
                    stocks[row['ticker']] = 0
                else:
                    new_n_units = stocks[row['ticker']] - \
                        row['tx_estimate'] / stock_price
                    stocks[row['ticker']] = max(0, new_n_units)
            portfolio_value_before_tx.append(value_before_tx)
            portfolio_value_after_tx.append(portfolio_value(stocks, date))
            tx_dates.append(date)
        except Exception as e:
            print('Ticker {0} gave exception: {1}'
                  .format(row['ticker'], e))
    return calculate_return(
        portfolio_value_before_tx,
        portfolio_value_after_tx,
        begin_date=min(rows[date_col]),
        end_date=max(rows[date_col]),
        tx_dates=tx_dates
    )

In [ ]:
senator_returns = []
senator_tx_growth = {}
senator_file_growth = {}
senator_names = set(clean_senators['full_name'])

In [ ]:
failed_senators = {}
print('{} senators total'.format(len(senator_names)))
for n in senator_names:
    print('Starting {}'.format(n))
    if n in senator_tx_growth:
        # Don't re-calculate for a given senator.
        continue
    try:
        tx_return = return_for_senator(
            clean_senators[clean_senators['full_name'] == n],
            date_col='tx_date')
        file_return = return_for_senator(
            clean_senators[clean_senators['full_name'] == n],
            date_col='file_date')
        senator_returns.append({
            'full_name': n,
            'tx_total_return': tx_return['portfolio_return'],
            'tx_cagr': tx_return['annual_cagr'],
            'file_total_return': file_return['portfolio_return'],
            'file_cagr': file_return['annual_cagr']
        })
        senator_tx_growth[n] = tx_return['growth']
        senator_file_growth[n] = file_return['growth']
    except Exception as e:
        print('Failed {0} with exception {1}'.format(n, e))
        failed_senators[n] = e

We look at the results to see the senators that outperformed the market.

In [ ]:
def plot_senator_growth(growth):
    """ Plot the senator's portfolio growth against the S&P 500. """
    df = pd.DataFrame(growth)
    plt.plot_date(df['date'], df['cum_growth'], '-')
    spy = load_for_ticker('SPY')
    spy = spy[(spy['date'] >= min(df['date']))
              & (spy['date'] <= max(df['date']))]
    spy_prices = spy['price']
    dates = spy['date']
    dates = dates.iloc[1:]
    spy_diff = np.cumprod(np.diff(spy_prices) / spy_prices[1:] + 1)
    plt.plot_date(dates, spy_diff, '-')
    plt.show()
    senator_growth = np.array(df['cum_growth'])
    print('Earliest date: {}'.format(min(df['date'])))
    print('Market return: {}'.format(
        spy_prices.iloc[-1] / spy_prices.iloc[0]))
    print('Senator return: {}'.format(
        senator_growth[-1] / senator_growth[0]))

In [ ]:
df = pd.DataFrame(senator_returns)
df = df[(df['tx_total_return'] > df['tx_cagr']) & df['tx_cagr'] > 0]
df.sort_values('tx_cagr')

In [ ]:
plot_senator_growth(senator_tx_growth['Angus S King, Jr.'])
clean_senators[clean_senators['full_name'] == 'Angus S King, Jr.']

## About this notebook

Author: Neel Somani, Software Engineer

Email: neel@berkeley.edu

Website: https://www.ocf.berkeley.edu/~neel/

Updated On: 2020-05-06